<a href="https://colab.research.google.com/github/mattiadido95/Haze-Fog-suppression/blob/main/pix2pixHD_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
!pip install dominate gdown

In [2]:
from google.colab import drive
import os
import gdown

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
folder_path = "drive/MyDrive/Haze-Fog-suppression"
os.chdir(folder_path)

# Download library for no reference metrics

In [ ]:
!git clone https://github.com/buyizhiyou/NRVQA.git

Cloning into 'NRVQA'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 166 (delta 43), reused 131 (delta 22), pack-reused 0
Receiving objects: 100% (166/166), 9.97 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (43/43), done.


# Utility

In [5]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd
import subprocess

# Save metrics in the pix2pixHD/results folder
def calculate_metrics(synth_folder, gt_folder, type):
    # Get the list of file names from the folders

    image_files = os.listdir(synth_folder)

    # Filter file names to remove those containing "input_label"
    filtered_image_files = [file for file in image_files if "input_label" not in file]

    # Initialize lists to store the results
    mse_scores = []
    ssim_scores = []
    psnr_scores = []

    piqe_scores = []
    niqe_scores = []

    # Iterate through the images and calculate metrics
    for image_file in filtered_image_files:
        # Load the images

        synth_image = cv2.imread(os.path.join(synth_folder, image_file))
        gt_image = cv2.imread(os.path.join(gt_folder, image_file[:4] + ".png"))

        # Resize the synthesized image to the same dimensions as the original image
        synth_image = cv2.resize(synth_image, (gt_image.shape[1], gt_image.shape[0]))

        # Calculate metrics using skimage
        mse = np.mean((gt_image - synth_image) ** 2)
        ssim_score = ssim(gt_image, synth_image, channel_axis=2)
        psnr_score = psnr(gt_image, synth_image)

        # Calculate metrics using piqe and niqe (high score means low quality)
        synth_array = synth_image / 255.0

        # Execute the command and capture the output
        completed_process = subprocess.run(['python', '/content/drive/MyDrive/Haze-Fog-suppression/NRVQA/test.py', '--mode', 'piqe', f'--path={os.path.join(synth_folder, image_file)}'], stdout=subprocess.PIPE, text=True)

        # Get the output from the CompletedProcess object
        output = completed_process.stdout

        # Extract the score from the output string
        score_line = [line for line in output.splitlines() if 'score:' in line][0]
        piqe_score = float(score_line.split(':')[-1])

        # Execute the command and capture the output
        completed_process = subprocess.run(['python', '/content/drive/MyDrive/Haze-Fog-suppression/NRVQA/test.py', '--mode', 'niqe', f'--path={os.path.join(synth_folder, image_file)}'], stdout=subprocess.PIPE, text=True)

        # Get the output from the CompletedProcess object
        output = completed_process.stdout

        # Extract the score from the output string
        score_line = [line for line in output.splitlines() if 'score:' in line][0]
        niqe_score = float(score_line.split(':')[-1])

        # Add the results to the lists
        mse_scores.append(mse)
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
        piqe_scores.append(piqe_score)
        niqe_scores.append(niqe_score)

    # Create the dataframe
    data = {
        'Image': filtered_image_files,  # Names of the generated image files
        'MSE': mse_scores,
        'SSIM': ssim_scores,
        'PSNR': psnr_scores,
        'PIQE': piqe_scores,
        'NIQE': niqe_scores
    }

    df = pd.DataFrame(data)

    # Calculate average values
    average_scores = df[['MSE', 'SSIM', 'PSNR', 'PIQE', 'NIQE']].mean()

    # Add the row with average values to the dataframe
    average_scores_df = pd.DataFrame([average_scores], columns=['MSE', 'SSIM', 'PSNR', 'PIQE', 'NIQE'])
    average_scores_df['Image'] = 'AVG'
    # Add an empty row between image results and average values
    df = pd.concat([df, pd.DataFrame(columns=df.columns)], ignore_index=True)
    df = pd.concat([df, average_scores_df], ignore_index=True)

    # Save the complete dataframe to a CSV file
    df.to_csv(type + '_result.csv', index=False)

def test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder, type):
  calculate_metrics(indoor_synth_folder, indoor_original_folder, type + "_indoor")
  calculate_metrics(outdoor_synth_folder, outdoor_original_folder, type + "_outdoor")


# Pix2PixHD download


In [ ]:
!git clone https://github.com/NVIDIA/pix2pixHD.git
os.chdir("pix2pixHD")

Cloning into 'pix2pixHD'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 55.68 MiB | 20.18 MiB/s, done.
Resolving deltas: 100% (156/156), done.
Updating files: 100% (115/115), done.
/content/drive/MyDrive/Haze-suppression/pix2pixHD


In [ ]:
os.makedirs("./checkpoints/label2city_1024p/")
os.chdir("./checkpoints/label2city_1024p/")

In [ ]:
url = 'https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download'
output = 'latest_net_G.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1h9SykUnuZul7J3Nbms2QGH1wa85nbN2-&export=download
To: /content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/checkpoints/label2city_1024p/latest_net_G.pth
100%|██████████| 732M/732M [00:11<00:00, 62.1MB/s]


'latest_net_G.pth'

In [ ]:
os.chdir("../..")

# Pix2PixHD Train


In [ ]:
%cd pix2pixHD/

/content/drive/.shortcut-targets-by-id/1MeMU7hWD5WVMk25vU3ch5Ii3csnSydwf/Haze-Fog-suppression/pix2pixHD


## Unique model for different level of fog

In [ ]:
#training on low resolution(512) only G1 (global net)
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
use_dropou

In [ ]:
#training on high resolution (1024) G1 and G2 (local+global) starting from epoch 100 of G1

#import torch
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
#torch.cuda.empty_cache()

!python train.py --netG local --continue_train --load_pretrain checkpoints/nebbia/ --label_nc 0 --no_instance --name nebbia --dataroot ./datasets/nebbia --resize_or_crop crop --fineSize 1024 --batchSize 2

------------ Options -------------
batchSize: 2
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 1024
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: checkpoints/nebbia/
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia
ndf: 64
nef: 16
netG: local
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_lo

## 3 GANS for each fog level

In [ ]:
#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia_low --dataroot ./datasets/nebbia_low --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia_low
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia_low
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False
us

In [ ]:
#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia_medium --dataroot ./datasets/nebbia_medium --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia_medium
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia_medium
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: Fa

KeyboardInterrupt: ignored

In [ ]:
#training on low resolution only G1
!python train.py --continue_train --label_nc 0 --no_instance --name nebbia_high --dataroot ./datasets/nebbia_high --resize_or_crop crop --fineSize 512 --batchSize 4

------------ Options -------------
batchSize: 4
beta1: 0.5
checkpoints_dir: ./checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/nebbia_high
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: nebbia_high
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: crop
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log: False


KeyboardInterrupt: ignored

# Pix2Pix test


## General model epoch 50




In [ ]:
#se non è stato eseguito il download
if os.getcwd() != "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD":
  os.chdir("pix2pixHD")

!python test.py --dataroot ./datasets/indoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --how_many 500 --which_epoch 50 --results_dir ./datasets/indoor/synth
!python test.py --dataroot ./datasets/outdoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --how_many 500 --which_epoch 50 --results_dir ./datasets/outdoor/synth

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/indoor
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 500
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: nebbia
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./datasets/indoor/synth
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: 50
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset]

In [ ]:
# Definisci i percorsi delle cartelle contenenti le immagini
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/synth/nebbia/test_50/images"  # Cartella con le immagini generate
indoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/gt"        # Cartella con le immagini originali
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/synth/nebbia/test_50/images"  # Cartella con le immagini generate
outdoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/gt"       # Cartella con le immagini originali
result_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results"

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder,os.path.join(result_folder,"50"))

## General model epoch 100

In [6]:
#se non è stato eseguito il download
if os.getcwd() != "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD":
  os.chdir("pix2pixHD")

!python test.py --dataroot ./datasets/indoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --how_many 500 --which_epoch 100 --results_dir ./datasets/indoor/synth
!python test.py --dataroot ./datasets/outdoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --how_many 500 --which_epoch 100 --results_dir ./datasets/outdoor/synth

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/indoor
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 500
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: nebbia
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./datasets/indoor/synth
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: 100
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDataset

In [7]:
# Definisci i percorsi delle cartelle contenenti le immagini
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/synth/nebbia/test_100/images"  # Cartella con le immagini generate
indoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/gt"        # Cartella con le immagini originali
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/synth/nebbia/test_100/images"  # Cartella con le immagini generate
outdoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/gt"       # Cartella con le immagini originali
result_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results"

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder,os.path.join(result_folder,"100"))

## General Model full train epoch 110

In [ ]:
# NON FUNZIONA PERCHE TRAIN.PY HA CORROTTO IL FILE LATEST_NET DURANTE IL TRAINING
!python test.py --netG local --dataroot ./datasets/indoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --how_many 500 --results_dir ./datasets/indoor/synth
!python test.py --netG local --dataroot ./datasets/outdoor --name nebbia --resize_or_crop none --label_nc 0 --no_instance --how_many 500 --results_dir ./datasets/outdoor/synth

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/indoor
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 500
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: nebbia
nef: 16
netG: local
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./datasets/indoor/synth
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ----------------
CustomDatasetDataLoader
dataset [AlignedDatas

In [ ]:
# define the paths of the folders containing the images
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/synth/nebbia/test_latest/images"  # Cartella con le immagini generate
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/synth/nebbia/test_latest/images"  # Cartella con le immagini generate

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder,os.path.join(result_folder,"latest"))

## Classifier + 3GAN epoch 50

### Classifier

In [ ]:
from keras.models import load_model
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# Load your classification model
classification_model = load_model("./classifier/dataset/models/InceptionV3/InceptionV3_128.h5")
# classification_model = load_model('./classifier/dataset/models/VGG16/VGG16_model.h5')

# Define class labels
class_labels = ['high_fog', 'low_fog', 'medium_fog', 'no_fog']

# Path to input images
base_path = "./pix2pixHD/datasets/"
context = ["indoor", "outdoor"]

# Output folder path
output_folder = "./pix2pixHD/results/"
for scene in context:
  input_folder = os.path.join(base_path, os.path.join(scene, "test_A"))
  # Loop through all images in the input folder
  for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg"):
      # Load the input image
      input_image = cv2.imread(os.path.join(input_folder, filename))

      # Resize the image if necessary
      width, height = 512, 512
      if input_image.shape[:2] != (height, width):
          input_image = cv2.resize(input_image, (width, height), interpolation=cv2.INTER_AREA)

      # Convert the image into a NumPy array
      image_array = np.array(input_image)

      # Normalize pixel values to the range [0, 1]
      image_array = image_array.astype('float32') / 255.0

      # Add a batch dimension if necessary
      image_batch = np.expand_dims(image_array, axis=0)

      # Perform image classification
      predicted_probabilities = classification_model.predict(image_batch)
      predicted_class = np.argmax(predicted_probabilities)
      image_class_label = class_labels[predicted_class]

      # Create the output folder if it does not exist
      output_class_folder = os.path.join(output_folder, os.path.join("fog3GAN", os.path.join(scene, os.path.join(image_class_label, "test_A"))))
      os.makedirs(output_class_folder, exist_ok=True)

      # Save the classified image to the output folder
      output_path = os.path.join(output_class_folder, filename)
      cv2.imwrite(output_path, input_image)


1/1 [==============================] - 0s 30ms/step


### Generate Images

In [ ]:
import subprocess

# If the download hasn't been executed
if os.getcwd() != "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD":
  os.chdir("pix2pixHD")

class_labels = ['high_fog', 'low_fog', 'medium_fog', 'no_fog']
context = ["indoor", "outdoor"]
base_path = "./results/fog3GAN"
checkpoints_dir = "./checkpoints"

for scene in context:
  for label in class_labels:
    dataset_path = os.path.join(base_path, os.path.join(scene, label))
    result_path = os.path.join(base_path, os.path.join(scene, os.path.join(label, "synth")))
    # Define the command to execute
    command = [
        'python', 'test.py',
        '--dataroot', dataset_path,
        '--checkpoints_dir', checkpoints_dir,
        '--name', label,
        '--resize_or_crop', 'none',
        '--label_nc', '0',
        '--no_instance',
        '--which_epoch', '50',
        '--results_dir', result_path,
        '--how_many', '500'
    ]

    # Execute the command using subprocess
    completed_process = subprocess.run(command, stdout=subprocess.PIPE, text=True)

    # Get the output from the CompletedProcess object
    output = completed_process.stdout

    # Print the output
    print(label)
    print(output)


nebbia_high
------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./results/nebbia3GAN/indoor/nebbia_high
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 500
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: nebbia_high
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: none
results_dir: ./results/nebbia3GAN/indoor/nebbia_high/synth
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: 50
-------------- End ----

### Evaluate images

In [ ]:
import shutil

class_labels = ['nebbia_high', 'nebbia_low', 'nebbia_medium', 'no_nebbia']
context = ["indoor","outdoor"]
base_path = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results/nebbia3GAN/"
for scene in context:
  for label in class_labels:
    if label=='no_nebbia':
      source = os.path.join(base_path, os.path.join(scene, os.path.join(label, "test_A")))
    else:
      source = os.path.join(base_path, os.path.join(scene, os.path.join(label, os.path.join("synth", os.path.join(label, os.path.join("test_50", "images"))))))
    dest = os.path.join(base_path, os.path.join(scene, "synth"))

    # get file list from the source folder
    elenco_file = os.listdir(source)

    if not os.path.exists(dest):
      os.makedirs(dest)

    # loop through the files and move them
    for file in elenco_file:
        percorso_file_origine = os.path.join(source, file)
        percorso_file_destinazione = os.path.join(dest, file)

        shutil.move(percorso_file_origine, percorso_file_destinazione)

print("Spostamento completato.")

Spostamento completato.


In [ ]:
# Define the paths to folders containing images
indoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results/fog3GAN/indoor/synth"  # Folder with generated images
outdoor_synth_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results/fog3GAN/outdoor/synth"  # Folder with generated images
indoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/indoor/gt"  # Folder with original images
outdoor_original_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/datasets/outdoor/gt"  # Folder with original images
result_folder = "/content/drive/MyDrive/Haze-Fog-suppression/pix2pixHD/results"

test_indoor_outdoor(indoor_synth_folder, indoor_original_folder, outdoor_synth_folder, outdoor_original_folder, os.path.join(result_folder, "3GAN"))
